# Download Europarl, the test data


In [ ]:
#Download all languages Europarl
import download
import threading

def maybe_download_and_extract(language_code="sv"):
    url = "http://www.statmt.org/europarl/v7/"

    # Create the full URL for the file with this data-set.
    data_url = url + language_code + "-en.tgz"
    data_dir = "data/europarl/" + language_code + "/"

    download.download(url=data_url, path=data_dir, kind="tar.gz", progressbar=False, replace=True)


def runInParallel():
  # We download the files in parallel to speed up and reduce the time needed.
  lang_codes = ["bg", "cs", "da", "de", "el", "es", "et", "fi", "fr", "hu", "it", "lv", "nl", "pl", "pt", "ro", "sk", "sl", "sv"]

  proc = []
  for code in lang_codes:
    thread = threading.Thread(target=maybe_download_and_extract, args=(code,))
    thread.start()
    proc.append(thread)
  for thread in proc:
    thread.join()

  print("done")

runInParallel()

# Download Corpora, the train data


In [ ]:
#Download all languages Corpora
import download
import threading

def downloading_corpus(url, dir, language_code):
    year=2018

    data_dir = "data/" + dir + "/" + language_code + "/"
    i = 1
    while i == 1:
        try:
            full_url = url + '_newscrawl_' + str(year) + '_10k.tar.gz'
            download.download(url=full_url, path=data_dir, kind="tar.gz", progressbar = False, replace=True, verbose=False)
            i=0
        except:
            year = year + 1

corpus_lang_codes = ["bul", "ces", "dan", "deu", "ell", "eng", "est", "fin", "fra", "hun", "ita", "lav", "lit", "nld", "pol", "por", "ron", "slk", "slv", "spa", "swe"]

proc = []
for code in corpus_lang_codes: 
    url = 'https://downloads.wortschatz-leipzig.de/corpora/' + code
    thread = threading.Thread(target=downloading_corpus, args=(url, "corpus", code,))
    thread.start()
    proc.append(thread)

for thread in proc:
    thread.join()

print("done")


In [9]:
import numpy as np
import scipy.io
import itertools # for generting all possible n-grams
import sklearn.metrics

#for visualization
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

n_g=2 # n-gram size
data = scipy.io.loadmat('languages_data.mat')
alphabet=(data['alphabet'])
alphabet=str(alphabet[0]) # turn into string
N_GRAMS=[p for p in itertools.product(alphabet, repeat=n_g)] # get all possible n-grams

# get data from the file
chop_size=int(data['chop_size'])
exm_size=int(data['exm_size'])
langLabels=(data['langLabels'])
testing=(data['testing'])
training=(data['training'])


# Print an example of a text in the training data
ind=3
print(langLabels[0][ind])
print(training[0][ind])
print (chop_size)

['nld']
[' juist om die ruimte te scheppen op deze site vindt u meer informatie over onze producten in zeeland kan de formatie van naaldwijk direct bovenop de formatie van maassluis liggen op zoek naar een unieke beleving sinds heeft hij een breed scala aan kennis en ervaring opgebouwd in de administratieve informatievoorziening de centrumlinkse kandidaat kreeg iets meer dan procent van de stemmen perfect internetworking solutions stelt de software van realaudio en realvideo gratis beschikbaar we kunnen nu opnieuw uit respect voor de kiezer geen steun verlenen aan heijenk over andere punten die de advocaat van louwes had aangedragen aan de hoge raad adviseert de advocaatgeneraal negatief die zijn in zijn ogen niet voldoende om de zaak ter herzien met de half miljard aan extra middelen moeten docenten worden bijgeschoold en ondersteund ook hierbij is de afsluiting zeer belangrijk om geen reinfectie te krijgen bij sylkamode vindt u een grote kast gevuld met allerlei kralen pijpjeskralen 

C:\Users\moell\AppData\Local\Temp\ipykernel_2140\3046671508.py:18: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  chop_size=int(data['chop_size'])
C:\Users\moell\AppData\Local\Temp\ipykernel_2140\3046671508.py:19: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  exm_size=int(data['exm_size'])


# Pre-processing 1.1 corpora


In [18]:
import os

def preprocess_text(content):
    # Replace "." with "\n" and ensure consistent line endings
    cleaned_content = content.replace('.', '\n').replace('\r\n', '\n').replace('\r', '\n')

    cleaned_content = ''.join(char if char.isalpha() or char.isspace() else ' ' for char in cleaned_content)

    # Remove extra whitespaces and normalize line endings
    cleaned_content = ' '.join(line.strip() for line in cleaned_content.splitlines())

    return cleaned_content

# Function to preprocess and save corpus
def preprocess_and_save_corpus(directory, corpus_lang_codes, max_chars=1000):
    for lang_code in corpus_lang_codes:
        input_file_path = os.path.join(directory, f"{lang_code}_newscrawl_10K-sentences.txt")
        output_file_path = os.path.join(directory, f"{lang_code}_newscrawl_10K-sentences_preprocd.txt")

        try:
            with open(input_file_path, 'r', encoding='utf-8') as file:
                lines = file.readlines()

            # Preprocessing logic for each line
            cleaned_lines = [preprocess_text(line) for line in lines]

            with open(output_file_path, 'w', encoding='utf-8') as file:
                file.write('\n'.join(cleaned_lines))

            print(f"Preprocessed and saved: {output_file_path}")

        except FileNotFoundError:
            print(f"Error: File '{input_file_path}' not found.")

# Example usage
corpus_directory = r'C:\Users\moell\build-unitime\d7041e\d7041e_lab1_JMEW\lab3\data\corpora'
corpus_lang_codes = ["bul", "ces", "dan", "deu", "ell", "eng", "est", "fin", "fra", "hun", "ita", "lav", "lit", "nld", "pol", "por", "ron", "slk", "slv", "spa", "swe"]

preprocess_and_save_corpus(corpus_directory, corpus_lang_codes)


TypeError: unbound method str.isalpha() needs an argument

# Pre-processing 1.1 europarl


In [ ]:
import os
from unidecode import unidecode

def preprocess_text_file(input_file, output_file, removal, max_rows=1000):
    try:
        # Read the content of the input file, limited to the first 1000 rows
        with open(input_file, 'r', encoding='utf-8') as file:
            content = file.read(max_rows)

        # Remove characters in the 'removal' array
        cleaned_content = ''.join(char for char in content if char not in removal)

        # Use unidecode to convert Unicode to ASCII
        cleaned_content = unidecode(cleaned_content)

        # Write the modified content back to the output file
        with open(output_file, 'w', encoding='utf-8') as file:
            file.write(cleaned_content)

        print(f"Text file '{input_file}' preprocessed and saved to '{output_file}'.")
    
    except FileNotFoundError:
        print(f"Error: File '{input_file}' not found.")

def preprocess_files_in_directory(directory, removal, max_rows=1000):
    for subdir, _, files in os.walk(directory):
        for file in files:
            input_file_path = os.path.join(subdir, file)
            output_file_name = f"pre-{os.path.basename(subdir)}_{file}"
            output_file_path = os.path.join(subdir, output_file_name)
            preprocess_text_file(input_file_path, output_file_path, removal, max_rows)
            print("Pre-processed", str(input_file_path))

# Example usage:
europarl_directory = r'C:\Users\moell\build-unitime\d7041e\d7041e_lab1_JMEW\lab3\data\europarl'
removal_characters = set("0123456789!@#$%^&*()_-+=<>?,./:;{}[]|`~\"'\\")  # Add other characters you want to remove
all_language_dirs  = ['bg', 'cs', 'da', 'de', 'el', 'es', 'et', 'fi', 'fr', 'hu', 'it', 'lv', 'nl', 'pl', 'pt', 'ro', 'sk', 'sl', 'sv']

preprocess_files_in_directory(europarl_directory, removal_characters)


# Remove preprocessed files


In [ ]:
import os

def remove_preprocessed_files(directory):
    for subdir, _, files in os.walk(directory):
        for file in files:
            if file.startswith("pre-"):
                file_path = os.path.join(subdir, file)
                os.remove(file_path)
                print(f"File '{file}' removed.")

# Example usage:
europarl_directory = r'C:\Users\moell\build-unitime\d7041e\d7041e_lab1_JMEW\lab3\data\europarl'
remove_preprocessed_files(europarl_directory)

# 2.0

In [ ]:
import numpy as np
encodingDict = {}

for i in letters:
    encodingDict[i] = np.random.choice([-1, 1], size=dimensions)

print(encodingDict)

In [ ]:
import os
import unidecode

testS = []
s_dict = {}
testingLabels = []
all_language_dirs  = ['bg', 'cs', 'da', 'de', 'el', 'es', 'et', 'fi', 'fr', 'hu', 'it', 'lv', 'nl', 'pl', 'pt', 'ro', 'sk', 'sl', 'sv']

letters = "abcdefghijklmnopqrstuvwxyz "
base_path = r"C:\Users\moell\build-unitime\d7041e\d7041e_lab1_JMEW\lab3\data\europarl"

for language in all_language_dirs:
    folder_path = os.path.join(base_path, language)
    for file_name in os.listdir(folder_path):
        if file_name.startswith("pre-"):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, mode="r", encoding="utf8") as data:
                i = 0
                for line in data:
                    dt = unidecode.unidecode(line).lower()
                    for x in dt:
                        if x not in letters:
                            dt = dt.replace(x, '')
                    ngrams = [dt[i:i + 2] for i in range(len(dt) - 1)]
                    for ngram in ngrams:
                        if s_dict.get(ngram) is None:
                            s_dict[ngram] = 1
                        else:
                            s_dict[ngram] += 1
                testS.append(s_dict)
                testingLabels.append(language)
                s_dict = {}
                i += 1
                if i >= 10:
                    break

print(testS)
print(testingLabels)

In [ ]:
import os
import unidecode

valid_characters = "abcdefghijklmnopqrstuvwxyz "
ngram_frequencies = {}
vector_dimensions = 100
ngram_length = 3
alphabet_size = 27
labels = []
training_data = []
base_path = r"C:\Users\moell\build-unitime\d7041e\d7041e_lab1_JMEW\lab3\data\europarl"
all_language_dirs = ['bg', 'cs', 'da', 'de', 'el', 'es', 'et', 'fi', 'fr', 'hu', 'it', 'lv', 'nl', 'pl', 'pt', 'ro', 'sk', 'sl', 'sv']

for language in all_language_dirs:
    folder_path = os.path.join(base_path, language)
    for file_name in os.listdir(folder_path):
        if file_name.startswith("pre-"):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, mode="r", encoding="utf8") as file:
                name_parts = file_name.split("_", 1)
                labels.append(name_parts[0])

                for line in file:
                    clean_line = unidecode.unidecode(line).lower()

                    for char in clean_line:
                        if char not in valid_characters:
                            clean_line = clean_line.replace(char, '')

                    ngrams = [clean_line[i:i + ngram_length] for i in range(len(clean_line) - 1)]

                    for ngram in ngrams:
                        if ngram not in ngram_frequencies:
                            ngram_frequencies[ngram] = 1
                        else:
                            ngram_frequencies[ngram] += 1

                training_data.append(ngram_frequencies)
                ngram_frequencies = {}


In [ ]:
import numpy as np
def transform_ngram_data(listS):
    listH = []
    for s in listS:
        h = np.zeros(dimensions)       
        for ngram in s.keys():
            n_i = np.random.choice([1], size = dimensions)
            s_i = s[ngram]
            j = 1
            for x in ngram:
                n_i = np.multiply(n_i, np.roll(encodingDict[x], j))
                j +=1
            h += s_i * n_i
        h = h/np.linalg.norm(h)
        listH.append(h)
    return(listH)

training_H = transform_ngram_data(training_data) 
testing_H = transform_ngram_data(testS)
print(training_H)
print(testing_H)


In [ ]:
predLabels = []

for test in testing_H:
    cos = []
    
    for training in training_H:
        cosine = np.dot(test, training) / (np.linalg.norm(test) * np.linalg.norm(training))
        cos.append(cosine)
    if cos:
        maxIndex = np.argmax(cos)
        predLabels.append(labels[maxIndex])
    else:
        # Handle the case where cos is empty (e.g., testing_H or training_H is empty)
        predLabels.append(None)

print(predLabels)


In [ ]:
import numpy as np

def encode_tri_gram(tri_gram):
    # Encode each element of the tri-gram to +1 or -1 randomly
    encoded_tri_gram = [1 if np.random.rand() > 0.5 else -1 for _ in tri_gram]
    return encoded_tri_gram

def map_to_hd_vectors(encoded_tri_gram, d1, d2):
    # Map the encoded tri-gram to HD vectors
    hd_vector_d1 = np.zeros(d1)
    hd_vector_d2 = np.zeros(d2)

    # Randomly select subsets of coordinates
    selected_coordinates_d1 = np.random.choice(d1, size=len(encoded_tri_gram), replace=False)
    selected_coordinates_d2 = np.random.choice(d2, size=len(encoded_tri_gram), replace=False)

    # Assign values to the selected coordinates
    hd_vector_d1[selected_coordinates_d1] = encoded_tri_gram
    hd_vector_d2[selected_coordinates_d2] = encoded_tri_gram

    return hd_vector_d1, hd_vector_d2

def process_text_file(file_path, d1, d2):
    with open(file_path, 'r') as file:
        text = file.read()

    tri_grams = [text[i:i+3] for i in range(len(text)-2)]
    
    hd_vectors_d1 = []
    hd_vectors_d2 = []

    for tri_gram in tri_grams:
        encoded_tri_gram = encode_tri_gram(tri_gram)
        hd_vector_d1, hd_vector_d2 = map_to_hd_vectors(encoded_tri_gram, d1, d2)
        hd_vectors_d1.append(hd_vector_d1)
        hd_vectors_d2.append(hd_vector_d2)

    return np.array(hd_vectors_d1), np.array(hd_vectors_d2)

# Example usage
file_path = 'your_text_file.txt'  # Replace with the path to your text file
d1 = 100
d2 = 1000

hd_vectors_d1, hd_vectors_d2 = process_text_file(file_path, d1, d2)

print("HD Vectors (d1):", hd_vectors_d1)
print("HD Vectors (d2):", hd_vectors_d2)


In [ ]:
# collect n-gram statitics for all training data 
TR_grams=np.zeros((len(N_GRAMS),langLabels.size), dtype='float') # initialize n-gram statistics for the  training

for i in range(langLabels.size):
    print(langLabels[0][i])

    buf_chop=training[0][i] #pick the current portion
    buf_chop=np.array2string(buf_chop) # get to string    
    buf_chop=(buf_chop[3:len(buf_chop)-2]) # get rid of artefacts like []
    
    for jj in range(len(buf_chop)-(n_g-1)):        
        ngc=buf_chop [jj:(jj+(n_g))] #pick current n-gram
        ngc1= tuple(ngc)
        ind_ngc=N_GRAMS.index(ngc1) # find index in  N_GRAMS
        TR_grams[ind_ngc,i]+=1 #increment the corresponding statisticss

    TR_grams[:,i]=TR_grams[:,i]/np.linalg.norm(TR_grams[:,i]) # normalize

print(TR_grams[:,0])    

In [ ]:
# collect n-gram statitics for all testing data 


GT = [None] * langLabels.size * exm_size   #ground truth  
PR = [None] * langLabels.size * exm_size   #predicton

for i in range(langLabels.size):    
    for j in range(exm_size): 
        TS_gram=np.zeros((len(N_GRAMS),1), dtype='float') # initialize n-gram statistics for the  test example
        buf_chop=testing[j][i] #pick the current portion
        buf_chop=np.array2string(buf_chop) # get to string    
        buf_chop=buf_chop[3:len(buf_chop)-2] # get rid of artefacts like []
        
        for jj in range(len(buf_chop)-(n_g-1)): 
            ngc=buf_chop [jj:(jj+(n_g))] #pick current n-gram
            ngc= tuple(ngc)
            ind_ngc=N_GRAMS.index(ngc) # find index in  N_GRAMS            
            TS_gram[ind_ngc,0]+=1 #increment the corresponding statisticss
            
        DP=np.dot(TS_gram.transpose(),TR_grams)
        ind=np.argmax(DP) #  index of predictd language
        PR[ i*exm_size +j ]=np.array2string(langLabels[0][ind])
        GT[ i*exm_size +j ]=np.array2string(langLabels[0][i]) # add ground truth

In [ ]:
# accuracy 
acc=0.0
for i in range(len(GT)):
   acc+=(PR[i]==GT[i])

acc=acc/len(GT) 
print(acc)

In [ ]:
langLabels[0][1][0].tolist()
Labels_arr=np.empty(21).astype(str)
for i in range(21):
    Labels_arr[i]=langLabels[0][i][0]
    
Labels_arr


In [ ]:
conf_mat= sklearn.metrics.confusion_matrix(GT, PR, labels=None, sample_weight=None)

#plot confusion matrix
%matplotlib inline
df_cm = pd.DataFrame(conf_mat, index = [i for i in Labels_arr], columns = [j for j in Labels_arr])
plt.figure(figsize = (18,18))
sn.heatmap(df_cm, annot=True)

In [ ]:
unicode_string = "Привет, мир!"  # This means "Hello, world!" in Russian

print(unicode_string)